## Get DOI
Obtains DOI links from Brenda Reactions.

Dependency: generate_brenda_reactions.ipynb output: brenda_rxns_incomplete.csv

In [1]:
import pandas as pd
from zeep import Client
import hashlib
import os
import pandas as pd
import time

# PUT IN EMAIL/PASSWORD HERE (must register for access):
## I've left in my own for now, but replace with the team one for ease
email = "mrunali@berkeley.edu"
pass_str = "brendaiscool"


wsdl = "https://www.brenda-enzymes.org/soap/brenda_zeep.wsdl"
password = hashlib.sha256(pass_str.encode("utf-8")).hexdigest()
client = Client(wsdl)

In [3]:
rxns = pd.read_csv("../Data_CSVs/brenda_rxns_incomplete.csv")
rxns.head()

,Unnamed: 0,literatureProducts,literatureSubstrates,organismSubstrates,commentarySubstrates,reversibility,organismProducts,products,commentaryProducts,substrates,ecNumber,pubmedId
0,0,NaN,695811.0,Thermus sp.,1% activity compared to cyclohexanone,?,NaN,? + NAD+,NaN,(-)-carvone + NADH + H+,1.1.1.1,18704396.0
1,1,NaN,695811.0,Thermus sp.,125% activity compared to cyclohexanol,r,NaN,(rac)-3-methylcyclohexanone + NADH + H+,163% activity compared to cyclohexanone,"(1S,3S)-3-methylcyclohexanol + NAD+",1.1.1.1,18704396.0
2,2,NaN,735498.0,Yokenella sp.,"41.5% of the activity with (2E)-but-2-enal, yi...",?,NaN,(2E)-2-methylpent-2-en-1-ol + NADP+,NaN,(2E)-2-methylpent-2-enal + NADPH + H+,1.1.1.1,24509923.0
3,3,NaN,735498.0,Yokenella sp.,"37.4% of the activity with (2E)-but-2-enal, yi...",?,NaN,"(2E)-3,7-dimethylocta-2,6-dien-1-ol + NADP+",NaN,"(2E)-3,7-dimethylocta-2,6-dienal + NADPH + H+",1.1.1.1,24509923.0
4,4,NaN,691867.0,Homo sapiens,substrate of isozyme ADH4,r,NaN,"(2E)-non-2-ene-1,4-diol + NAD+",NaN,(2E)-4-hydroxynon-2-enal + NADH + H+,1.1.1.1,18505683.0


In [4]:
print("{} unique PMIDs and {} total rows".format(len(rxns["pubmedId"].unique()), len(rxns)))


103948 unique PMIDs and 459935 total rows


In [5]:
print("{} 0.0 PMID values which is {}% of the total".format(len(rxns[rxns["pubmedId"] == 0]), 100*len(rxns[rxns["pubmedId"] == 0])/len(rxns)))
#

59847 0.0 PMID values which is 13.012056051398567% of the total


In [6]:
PMrxns = rxns[rxns["pubmedId"] != 0].drop("Unnamed: 0", axis = 1)
PMrxns.head()

,literatureProducts,literatureSubstrates,organismSubstrates,commentarySubstrates,reversibility,organismProducts,products,commentaryProducts,substrates,ecNumber,pubmedId
0,NaN,695811.0,Thermus sp.,1% activity compared to cyclohexanone,?,NaN,? + NAD+,NaN,(-)-carvone + NADH + H+,1.1.1.1,18704396.0
1,NaN,695811.0,Thermus sp.,125% activity compared to cyclohexanol,r,NaN,(rac)-3-methylcyclohexanone + NADH + H+,163% activity compared to cyclohexanone,"(1S,3S)-3-methylcyclohexanol + NAD+",1.1.1.1,18704396.0
2,NaN,735498.0,Yokenella sp.,"41.5% of the activity with (2E)-but-2-enal, yi...",?,NaN,(2E)-2-methylpent-2-en-1-ol + NADP+,NaN,(2E)-2-methylpent-2-enal + NADPH + H+,1.1.1.1,24509923.0
3,NaN,735498.0,Yokenella sp.,"37.4% of the activity with (2E)-but-2-enal, yi...",?,NaN,"(2E)-3,7-dimethylocta-2,6-dien-1-ol + NADP+",NaN,"(2E)-3,7-dimethylocta-2,6-dienal + NADPH + H+",1.1.1.1,24509923.0
4,NaN,691867.0,Homo sapiens,substrate of isozyme ADH4,r,NaN,"(2E)-non-2-ene-1,4-diol + NAD+",NaN,(2E)-4-hydroxynon-2-enal + NADH + H+,1.1.1.1,18505683.0


For PM reactions, we can download from SciHub using PMID. For non-PubMed references, we need to get remainder of reference information to get the DOI using CrossRef so we can download the the papers from SciHub. 

In [7]:
nonPMrxns = rxns[rxns["pubmedId"] == 0].drop("Unnamed: 0", axis = 1).drop("pubmedId", axis = 1)
nonPMrxns.head()

,literatureProducts,literatureSubstrates,organismSubstrates,commentarySubstrates,reversibility,organismProducts,products,commentaryProducts,substrates,ecNumber
10,NaN,659773.0,Neurospora crassa,NaN,NaN,NaN,"(4S,6S)-5,6-dihydro-4-hydroxy-6-methyl-4H-thie...",NaN,"(6S)-5,6-dihydro-6-methyl-4H-thieno[2,3b]thiop...",1.1.1.1
12,NaN,285628.0,Camellia sinensis,43.2% of the activity with ethanol,?,NaN,(E)-hex-2-en-1-one + NADH,NaN,(E)-hex-2-en-1-ol + NAD+,1.1.1.1
19,NaN,688442.0,Devosia riboflavina,55% of activity with N-benzyl-3-pyrrolidinol,?,NaN,2-butanone + NADH + H+,NaN,(R)-2-butanol + NAD+,1.1.1.1
20,NaN,688442.0,Devosia riboflavina KNK10702,55% of activity with N-benzyl-3-pyrrolidinol,?,NaN,2-butanone + NADH + H+,NaN,(R)-2-butanol + NAD+,1.1.1.1
73,NaN,285628.0,Camellia sinensis,9.7% of the activity with ethanol,?,NaN,(Z)-hex-2-en-1-one + NADH,NaN,(Z)-hex-2-en-1-ol + NAD+,1.1.1.1


In [8]:
len(nonPMrxns["literatureSubstrates"].unique())

12064

In [9]:
len(nonPMrxns["literatureSubstrates"])

59847

In [10]:
# use getReferenceById to fill in the missing rows if possible
# no need to run again, we created a reference dictionary as a pickle file that can be imported
# Takes ~30 minutes to run (1640/60 = 27 minutes)

# nonPMlitIDs = nonPMrxns.literatureSubstrates.unique() 
# count1 = 0 
# t0 = time.time()
# refDict = {}
# connectionErrorsIds = {}

# for li in nonPMlitIDs:
#     count1 = count1 + 1
#     if (count1 % 500) == 0:
#         print(str(count1) + " out of " + str(len(nonPMlitIDs)))
#         print(time.time() - t0)
#         print('\n')
#     try:
#         parameters = (email,password,li)
#         data = client.service.getReferenceById(*parameters) 
#         refDict[li] = data
#     except Exception as e:
#         connectionErrorsIds[li] = e
#         continue

# print(time.time() - t0)

# refDict

In [11]:
try:
    import cPickle as pickle
except ImportError:  # Python 3.x
    import pickle

In [17]:
with open('./get_doi/refDict.p', 'rb') as fp:
    refDict = pickle.load(fp)

Use reference dictionary to get information needed for CrossRef to get DOIs as a string. 

In [108]:
import lxml.etree as ET
import urllib
import re

def get_doi(li):
    DOIerrors = {}
    data = refDict[li]
#     for i in (["journal", "volume", "year","pages"]): # why is this here?
    print(data)
    
    journal = " ".join(data["journal"].split())
    volume = "".join(data["volume"].split())
    year = "".join(data["year"].split())
    start_page = "".join(data["pages"].split()).split("-")[0]
    first_author_last_name = "".join(data["authors"].split(";")[0].split(",")[0].split())
    journal_title = journal.replace(" ", '%20')
    params = (first_author_last_name, journal_title, volume, start_page, year)
    url = "https://doi.crossref.org/openurl?pid=mrunali@berkeley.edu&aulast={}&title={}&volume={}&spage={}&date={}&noredirect=true".format(*params)

    data = urllib.request.urlopen(url).read()
    print(data)
    print(url)
#     root = ET.fromstring(data)
#     matches = root.findall('.//doi')[1]
    data = data.decode("utf-8")
    
#     print(li)

    if (data.find('unresolved') != -1):
        DOIerrors[li] = data
        return
    else:
        try:
            doi = re.findall('(<doi.*>)(.*)<\/doi>', data)[0][1]
#             return "https://sci-hub.tw/{}".format(doi)
            return doi
        except Exception as e:
            DOIerrors[li] = data
            print(li)
    

Trying out ONE key (brenda ref)

In [111]:
get_doi(659773)

{
    'authors': 'Blacklock, T.J.; Sohar, P.; Butcher, J.W.; Lamanec, T.; Grabowski, E.J.J.',
    'title': 'An enantioselective synthesis of the topically-active carbonic anhydrase inhibitor MK-0507:5,6-dihydro-(s)-4-(ethylamino)-(s)-6-mehtyl-4H-thieno[2,3-beta]thiopyran-2-sulfonamide 7,7-dioxide hydrochloride',
    'journal': 'J. Org. Chem.',
    'volume': '58',
    'pages': '1672-1679',
    'year': '1993',
    'pubmedId': '0',
    'textmining': '0'
}
b'<?xml version="1.0" encoding="UTF-8"?>\n<crossref_result xmlns="http://www.crossref.org/qrschema/2.0" version="2.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.crossref.org/qrschema/2.0 https://www.crossref.org/schema/crossref_query_output2.0.xsd"><query_result><head><doi_batch_id>none</doi_batch_id></head><body><query key="555-555" status="resolved" fl_count="52"><!--score=1--><doi type="journal_article">10.1021/jo00059a013</doi><issn type="print">0022-3263</issn><issn type="electronic">1520-69

'10.1021/jo00059a013'

In [110]:
import math

test_ref = math.floor(list(refDict.keys())[3])
test_title = refDict[test_ref]['title'].strip()
print(test_ref)
print('Purification and characterization of NAD-dependent alcohol dehydrogenase and NADH-dependent 2-oxoaldehyde reductase from parsley'.replace(" ", '%20'))

743908
Purification%20and%20characterization%20of%20NAD-dependent%20alcohol%20dehydrogenase%20and%20NADH-dependent%202-oxoaldehyde%20reductase%20from%20parsley


In [102]:
doi_map = {}
for li in refDict.keys():
    doi_temp = get_doi(li)
    doi_map[li] = doi_temp
    print(doi_temp)

nonPMrxns['literatureSubstrates'].map(doi_map)
nonPMrxns

659773.0
10.1021/jo00059a013
285628.0
10.1080/00021369.1972.10860516
688442.0
10.1016/j.molcatb.2007.10.017
743908.0
10.1021/acscatal.7b03809
285627.0
None
285630.0
10.1016/0167-4838(82)90369-7
745570.0
10.1016/j.molcatb.2008.05.012
285639.0
10.1016/0141-0229(86)90084-0
658565.0
10.1016/S0141-0229(96)00177-9
285583.0
None


KeyboardInterrupt: 

In [ ]:
len(DOIerrors)

In [ ]:
https://github.com/zaytoun/scihub.py